In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from datetime import datetime as dt
from db.db_operations import execute_db_operations

from generators.full_generators import (
    create_company_data
)

company_name = "Lego"

In [2]:
data = create_company_data(company_name=company_name, save_to_csv=True)

Generating data for company: Lego...
Using existing context report for Lego.
Context data for Lego generated: {'company_name': 'Lego', 'count_employee': 550, 'count_department': 25, 'count_customer': 55, 'count_product': 180, 'count_procurement': 140, 'count_service': 140, 'count_account': 60, 'count_vendor': 75, 'estimated_product': 105000000, 'estimated_service': 40000000, 'estimated_overhead': 442000000, 'estimated_revenue': 649000000}
Generating dimensions for company: Lego.
This usually takes 2-5 mins.

=== Attempt 1 ===
✔ Roles and Names generated.
✔ Procurement data generated.
✔ Services data generated.
✔ Products data generated.
✔ Accounts data generated.
✔ Customers data generated.
✔ Departments data generated.
✔ Vendors data generated.
✔ Payroll data generated.
✔ All table lengths validated.
✔ All CSVs saved to: data/outputdata

 * Semantic mapping started * :
Time estimate: 3-5 minutes
✔ Procurement mapping done!
✔ Service mapping done!
✔ Product mapping done!
✔ All mapping 

In [3]:
now = dt.now()
version_tag = (
    company_name.lower()
    + "_date_" + now.strftime("%m%d")
    + "_time_" + now.strftime("%H%M%S")
)
execute_db_operations(version_tag)

[INFO] Inserting 25 rows into dim_department using to_sql...
[INFO] Inserting 55 rows into dim_customer using to_sql...
[INFO] Inserting 180 rows into dim_product using to_sql...
[INFO] Inserting 60 rows into dim_account using to_sql...
[INFO] Inserting 140 rows into dim_procurement using to_sql...
[INFO] Inserting 140 rows into dim_service using to_sql...
[INFO] Inserting 14 rows into dim_payline using to_sql...
[INFO] Inserting 75 rows into dim_vendor using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)


[INFO] Inserting 550 rows into dim_employee using to_sql...
[INFO] Inserting 554400 rows into fact_payroll using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain 

[INFO] Inserting 49920 rows into fact_general_ledger using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain 

[INFO] Inserting 33355 rows into fact_budget using to_sql...
